## Library to use in main file

### Importing necessary libraries

In [1]:
from datetime import datetime as dt
from yahoo_fin.stock_info import get_data

import numpy as np
import pandas as pd
from datetime import date

from scipy.optimize import minimize
from scipy.stats import norm
import matplotlib.pyplot as plt

### Function to Fetch Price and Returns DataFrame

In [2]:
def Prices_Return_Df(ticker,from_date,riskFreeRate): #This function fetches prices from yahoo finance to create price and return dataframe 
    now=date.today().strftime("%d/%m/%Y")   
    prices=pd.DataFrame()
    for x in ticker: 
        try:            
            data= get_data(x, start_date=from_date, end_date=now, index_as_date = True, interval="1d").close
            year=data.index[0].strftime("%Y")
            month=data.index[0].strftime("%m")
            act_date=dt.strptime(from_date, "%m/%d/%Y")
            if month == act_date.strftime("%m") and year == act_date.strftime("%Y"): #checking start month and year of price before appending with exisiting dataframe
                prices[x] = data                
                print("Successfully Fetched","|",x)
            else:
                print("Insufficient data",x)
        except:
            print("Price Fetch Error:Could Be Invalid Symbol",x)               
    prices_df=prices.dropna()
    returns_df = prices.pct_change()-(riskFreeRate/252) #calculating percent change / rate of return
    returns_df=returns_df.dropna()
    print(len(prices),"|","Total length of data Used for Analysis")
    return prices_df , returns_df

### Structure of Library

<img src="portfolio_library.png">

### Metric Class to obtain necessary metrics for calculation and optimization (Building Blocks - Foundation)

In [3]:
class Metrics:
    def __init__(self,returns_df,invested_capital):
        self.returns_df=returns_df #Get returns dataframe
        self.invested_capital=invested_capital #Get Invested Capital
        self.ticker=returns_df.columns #Gets list of obtained ticker

    
    def covMatrix(self): #Derive Covariance Matrix from the returns dataframe.
        df=self.returns_df.values
        covMatrix = np.cov(df,rowvar=False) #obtain covariance
        return covMatrix
    
    def meanReturns(self): #Derive Mean Returns of each Instrument/Symbol from the returns dataframe.
        df=self.returns_df.values
        meanReturns = np.mean(df,axis=0) #mean return of individual stocks
        return meanReturns
    
    def portfolio_return(self,weights): #Derive Portfolio Return by using meanReturns function.
        meanReturns=self.meanReturns()
        returns = np.sum(meanReturns*weights)*252 #sum of all returns of stocks based on our weigtage
        return returns
    
    def portfolio_variance(self,weights): #Derive Portfolio Variance by using covMatrix function.
        covMatrix=self.covMatrix()        
        variance=np.dot(weights.T,np.dot(covMatrix, weights))*252 #Portfolio Variance Calculation
        return variance
        
    def portfolio_stdev(self,weights): #Derive standard-deviation or volatility of the portfolio by using portfolio_variance function.
        stdev=np.sqrt(self.portfolio_variance(weights)) #Portfolio volatility calculation
        return stdev
    
    def portfolio_VaR(self,weights,confidence,days): #Derive VaR of the portfolio by using portfolio_stdev function.
        volatility=self.portfolio_stdev(weights)
        stress_event=norm.ppf(confidence)
        VaR=self.invested_capital*volatility*stress_event*np.sqrt(days/252)# VaR calculation
        VaR_Percent=(VaR/self.invested_capital)
        return VaR_Percent
        
        
    def risk_weights(self,weights): #Derive risk allocation of individual stocks of the portfolio by using covMatrix and portfolio_variance functions.
        covMatrix=self.covMatrix() 
        variance=self.portfolio_variance(weights)
        cw=np.dot(covMatrix, weights)
        weights_risk=[]
        for x in range(0,len(cw)):
            risk=((cw[x]*weights[x])/variance)*252 #risk allocation calculation based on weight
            weights_risk.append(risk)
        return weights_risk
    
    
    def Balance_Array(self,weights): #Derive series Balance array/List by using returns dataframe
        try:
            df=self.returns_df.copy()
            modified_returns=df*weights       
            df['portfolio_return']=modified_returns.sum(axis = 1, skipna = True)
            balance_array=[]
            for x in range(0,len(df)):
                if x==0:
                    balance_array.append(self.invested_capital)
                else:
                    prev_ind=len(balance_array)-1
                    prev_bal=balance_array[prev_ind]
                    balance_array.append(df['portfolio_return'].iloc[x]*prev_bal+prev_bal)#running balance calculation
            return balance_array,np.array(df.index)
        except:
            return "please check the dataframe"
        
    def Max_Drawdown(self,weights): #Derive Maximum Drawdown by using Balance_Array function
        balance,dti=self.Balance_Array(weights)
        max_balance=[]
        drawdown=[]
        try:
            for x in range(0,len(balance)): 
                if x==0:
                    max_balance.append(balance[0]) 
                else:
                    max_balance.append(max(balance[0:x])) 
                    
            for x in range(0,len(balance)): 
                temp_data=balance[x]/max_balance[x]-1 #running drawdown calculation
                drawdown.append(temp_data)
                
            max_drawdown=abs(min(drawdown))
            return max_drawdown
        except:
            return "please check the dataframe"
        
    def Balance_Graph(self,weights): #Derive Performance Graph by using Balance_Array function
        balance,dti=self.Balance_Array(weights)
        plt.figure(figsize=(10, 6))
        plt.plot(dti,balance)  
        plt.title("Equity Curve")        
        plt.xlabel("Date")
        plt.ylabel("Balance") 
        plt.show()   
    
    def All_Balance_Graph(self,eq_weights,rc_weights,mv_weights): #Derive Performance Graph
        eq_balance,dti=self.Balance_Array(eq_weights)
        rc_balance,dti=self.Balance_Array(rc_weights)
        mv_balance,dti=self.Balance_Array(mv_weights)
        plt.figure(figsize=(10, 6))
        plt.plot(dti,eq_balance) 
        plt.plot(dti,rc_balance)
        plt.plot(dti,mv_balance) 
        plt.legend(["Equally Weighted", "Equal Risk Contribution","Mean Variance"])
        plt.title("Equity Curve")        
        plt.xlabel("Date")
        plt.ylabel("Balance")        
        plt.show()  

### Optimal_Requirement Class is a child class of metrics which is used for optimizing them to reach desired results (Intermediate Floor)

In [4]:
class Optimal_Requirements(Metrics):
    
    def deviation(self,weights): #Derive Deviation from equally allocated risk and portfolio allocated risk
        meanReturns=self.meanReturns()
        fixed_risk = np.empty(len(meanReturns))
        fixed_risk.fill((1/len(meanReturns)))           
        risk_allocated=self.risk_weights(weights)        
        dev_list=np.abs(np.subtract(fixed_risk,risk_allocated)) #subtracting fixed risk and allocated risk to get deviation list
        deviation_value=sum(dev_list) #sum of all elements in deviation list
        return deviation_value
    
    def negativeSR(self,weights): #Derive Negative Sharpe Ratio of the portfolio by using portfolio_return and portfolio_stdev functions.
        returns=self.portfolio_return(weights)
        stdev=self.portfolio_stdev(weights)
        nSR =- (returns)/stdev
        return nSR

### Portfolio_Weights Class is a child class of Optimal_Requirements which is used for obtaining optimized weights (Top Floor)

In [5]:

class Portfolio_Weights(Optimal_Requirements): 
        
    def Equally_Weighted(self): #Derive Equally weighted array/list
        meanReturns=self.meanReturns()
        numAssets=len(meanReturns)
        weights = np.empty(numAssets)#this will create empty list with numAsset length
        weights.fill((1/numAssets))#fill the list with  1/numAsset value
        weights_df=pd.DataFrame()
        weights_df['ticker']=self.ticker
        weights_df['weights']=weights
        return weights_df
    
    def Risk_Contributed_Optimized_Weights(self,constraintSet=(0,1)): #Derive Risk Contributed weighted array/list
        "Minimize deviation ,by altering the weights of the portfolio"
        meanReturns=self.meanReturns()       
        numAssets = len(meanReturns)

        constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1}) #contraint that sum of weights should be equal to 1
        bound = constraintSet #boundaries of each weight has to be between 0 - 1 
        bounds = tuple(bound for asset in range(numAssets))
        result = minimize(self.deviation, numAssets*[1./numAssets],
                            method='SLSQP', bounds=bounds, constraints=constraints) #minimizing deviation by using SLSQP Algo
        weights=np.around(result['x'],4)
        weights_df=pd.DataFrame()
        weights_df['ticker']=self.ticker
        weights_df['weights']=weights
        return weights_df  
    
    def meanVariance_Optimized_Weights(self, constraintSet=(0,1)): #Derive Mean Variance optimised weighted array/list
        "Minimize the negative SR, by altering the weights of the portfolio"
        meanReturns=self.meanReturns()
        numAssets = len(meanReturns)
  
        constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
        bound = constraintSet
        bounds = tuple(bound for asset in range(numAssets))
        result = minimize(self.negativeSR, numAssets*[1./numAssets],
                            method='SLSQP', bounds=bounds, constraints=constraints) #minimizing negativeSR by using SLSQP Algo
        weights=np.around(result['x'],4)
        weights_df=pd.DataFrame()
        weights_df['ticker']=self.ticker
        weights_df['weights']=weights
        return weights_df
 